In [15]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt
import json
import ast


Cargamos las bases de datos

In [16]:
dir_data = "../airbnb_final_csv/"
calendar = pd.read_csv(f"{dir_data}calendar_fin.csv")
listings = pd.read_csv(f"{dir_data}listings_fin.csv")
neighbourhoods = pd.read_csv(f"{dir_data}neighbourhoods_fin.csv")
reviews = pd.read_csv(f"{dir_data}reviews_fin.csv")
reviews_gz = pd.read_csv(f"{dir_data}reviews_gz_fin.csv")
listings_gz = pd.read_csv(f"{dir_data}listings_gz_fin.csv")

/tmp/ipykernel_6771/473357094.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv(f"{dir_data}calendar_fin.csv")
/tmp/ipykernel_6771/473357094.py:7: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_gz = pd.read_csv(f"{dir_data}listings_gz_fin.csv")


Procesamos los datos para poder cargarlos a la base de datos. Empezamos con dos datos de listings_gz

In [17]:
# Proceso las columnas que deberian que son booleanas y tienen "f" y "t" como valor:

listings_gz = listings_gz.drop_duplicates()

listings_gz["host_is_superhost"] = listings_gz["host_is_superhost"].replace({"f":"false", "t":"true", })
listings_gz["host_has_profile_pic"] = listings_gz["host_has_profile_pic"].replace({"f":"false", "t":"true", })
listings_gz["host_identity_verified"] = listings_gz["host_identity_verified"].replace({"f":"false", "t":"true", })
listings_gz["has_availability"] = listings_gz["has_availability"].replace({"f":"false", "t":"true", })
listings_gz["instant_bookable"] = listings_gz["instant_bookable"].replace({"f":"false", "t":"true", })

# Tratamos los valores nulos:

# Calendar_updated podriamos eliminarlo ya que no tiene ningun valor, todos son valores nulos
listings_gz["calendar_updated"].fillna(0).astype(int)


0        0
1        0
2        0
3        0
4        0
        ..
93596    0
93597    0
93598    0
93599    0
93600    0
Name: calendar_updated, Length: 93601, dtype: int64

Ahora vamos con las variables numericas

In [20]:
# Empezamos quitandole el simbolo del porcentaje y diviendiendo entre 100 a las variables host_response_rate y a host_
# Para ello utilizamos una funcion:

def conversion_porcentajes(name):
    name = str(name).replace("%","")
    name = float(name)/100
    return name

listings_gz["host_response_rate"] = listings_gz["host_response_rate"].apply(conversion_porcentajes)
listings_gz["host_acceptance_rate"] = listings_gz["host_acceptance_rate"].apply(conversion_porcentajes)

# Convertimos algunos float en int, antes quitamos valores nulos igualando a 0 (ya que solo hay 6 valores nulos)

listings_gz["host_listings_count"] = listings_gz["host_listings_count"].fillna(0).astype(int)
listings_gz["host_total_listings_count"] = listings_gz["host_total_listings_count"].fillna(0).astype(int)

# Nos aseguramos de que host_verifications este en formato json, pero antes manejamos los valores nulos

listings_gz['host_verifications'] = listings_gz['host_verifications'].fillna('[ ]')
listings_gz['host_verifications'] = listings_gz['host_verifications'].apply(lambda x: json.dumps(ast.literal_eval(x)))

ValueError: malformed node or string: nan

In [14]:
# Los nombres de las columnas correctas, EN ORDEN
proc_host_column_names_new = ['id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate','acceptance_rate', 'is_superhost', 
                            'thumbnail_url', 'picture_url', 'neighbourhood','listings_count', 'total_listings_count', "verifications", 'has_profile_pic','identity_verified', 
                            'calculated_host_listings_count','calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                            'calculated_host_listings_count_shared_rooms']

# Creamos el dataframe, EN EL ORDEN ANTERIOR
proc_hosts = listings_gz[["host_id", "host_url", "host_name", "host_since", "host_location", "host_about", "host_response_time", "host_response_rate", "host_acceptance_rate", 
                        "host_is_superhost", "host_thumbnail_url", "host_picture_url", 'host_neighbourhood', "host_listings_count",  "host_total_listings_count", "host_verifications",
                        "host_has_profile_pic", "host_identity_verified", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]]

#Con esto renombro las columnas
proc_hosts = proc_hosts.set_axis(proc_host_column_names_new, axis=1)
proc_hosts = proc_hosts.drop_duplicates(subset="id")

# Esto es una prueba para ver si sale bien
proc_hosts.to_csv('../proc_airbnb/proc_host.csv', index=False)

In [11]:
# Create dataframes from SQL tables
# Calendar table
proc_calendar = pd.DataFrame(columns=[
    'listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'
])

# Property type table
proc_property_type = pd.DataFrame(columns=['id', 'type'])

# Listings table
proc_listings = pd.DataFrame(columns=[
    'id', 'name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'city', 'latitude', 'longitude',
    'room_type', 'property_type', 'price', 'minimum_nights', 'maximum_nights', 'license'
])

# Listings description table
proc_listings_description = pd.DataFrame(columns=[
    'listing_id', 'description', 'picture_url', 'accommodates', 'bathrooms', 'bathrooms_text',
    'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'
])

# Listings amenities table
proc_listings_amenities = pd.DataFrame(columns=['listing_id', 'amenity'])

# Listings availability table
proc_listings_availability = pd.DataFrame(columns=[
    'listing_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
    'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'instant_bookable'
])

# Listings reviews table
proc_listings_reviews = pd.DataFrame(columns=[
    'listing_id', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'reviews_per_month'
])

# Neighborhood groups table
proc_neighborhood_groups = pd.DataFrame(columns=['id', 'name', 'neighbourhood_group_cleansed'])

# Neighborhoods table
proc_neighborhoods = pd.DataFrame(columns=[
    'id', 'name', 'neighborhood_cleansed', 'neighborhood_group', 'overview', 'city'
])

# Countries table
proc_countries = pd.DataFrame(columns=['iso_3166_1', 'name'])

# Cities table
proc_cities = pd.DataFrame(columns=['id', 'name', 'country_code'])

# Hosts table
proc_hosts = pd.DataFrame(columns=[
    'id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate',
    'acceptance_rate', 'is_superhost', 'thumbnail_url', 'picture_url', 'neighborhood',
    'listings_count', 'total_listings_count', 'verifications', 'has_profile_pic',
    'identity_verified', 'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
])


# Users table
proc_users = pd.DataFrame(columns=['id', 'name'])

# Reviews table
proc_reviews = pd.DataFrame(columns=['id', 'listing_id', 'date', 'reviewer_id', 'comments'])

In [12]:
# Create a dictionary to map information from listings to the new dataframes
listings_info = {
    'proc_listings_description': {
        'listings_id': listings['id'],
        'description': listings['description'],
        'picture_url': listings['picture_url'],
        'accommodates': listings['accommodates'],
        'bathrooms': listings['bathrooms'],
        'bathrooms_text': listings['bathrooms_text'],
        'bedrooms': listings['bedrooms'],
        'beds': listings['beds'],
        'minimum_minimum_nights': listings['minimum_minimum_nights'],
        'maximum_minimum_nights': listings['maximum_minimum_nights'],
        'minimum_maximum_nights': listings['minimum_maximum_nights'],
        'maximum_maximum_nights': listings['maximum_maximum_nights'],
        'minimum_nights_avg_ntm': listings['minimum_nights_avg_ntm'],
        'maximum_nights_avg_ntm': listings['maximum_nights_avg_ntm']
    },
    'proc_listings': {
        'id': listings['id'],
        'name': listings['name'],
        'host_id': listings['host_id'],
        'neighbourhood_group': listings['neighbourhood_group_cleansed'],
        'neighbourhood': listings['neighbourhood_cleansed'],
        'city': listings['city'],
        'latitude': listings['latitude'],
        'longitude': listings['longitude'],
        'room_type': listings['room_type'],
        'property_type': listings['property_type'],
        'price': listings['price'],
        'minimum_nights': listings['minimum_nights'],
        'maximum_nights': listings['maximum_nights'],
        'license': listings['license']
    }, 
    'proc_listings_amenities': {
        'listing_id': listings['id'],
        'amenity': listings['amenities']
    }
}
# Map the dictionary to the new dataframes
for key, value in listings_info.items():
    for col, series in value.items():
        if key == 'proc_listings_description':
            proc_listings[col] = series
        elif key == 'proc_listings':
            proc_listings[col] = series
        elif key == 'proc_listings_amenities':
            proc_listings_amenities[col] = series

# Create a dictionary to map information from calendar to the new dataframes
calendar_info = {
    'proc_calendar': {
        'listing_id': calendar['listing_id'],
        'date': calendar['date'],
        'available': calendar['available'],
        'price': calendar['price'],
        'adjusted_price': calendar['adjusted_price'],
        'minimum_nights': calendar['minimum_nights'],
        'maximum_nights': calendar['maximum_nights']
    }
}
# Map the dictionary to the new dataframes
for key, value in calendar_info.items():
    for col, series in value.items():
        if key == 'proc_calendar':
            proc_calendar[col] = series

neighbourhoods_info = {
    'proc_neighborhoods': {
        'neighborhood_cleansed': neighbourhoods['neighbourhood'],
        'neighborhood_group': neighbourhoods['neighbourhood_group'],
        'city': neighbourhoods['city'],
    }
}

# Map the dictionary to the new dataframes
for key, value in neighbourhoods_info.items():
    for col, series in value.items():
        if key == 'proc_neighborhoods':
            proc_neighborhoods[col] = series
    

KeyError: 'description'

In [31]:
proc_cities['id'] = range(1, len(cities) + 1)
proc_cities['name'] = cities
proc_cities['country_code'] = 'ES'

In [32]:
proc_countries['iso_3166_1'] = 'ES'
proc_countries['name'] = 'Spain'

In [33]:
proc_neighborhood_groups['neighbourhood_group_cleansed'] = neighbourhoods['neighbourhood_group'].unique()
proc_neighborhood_groups['id'] = range(1, len(proc_neighborhood_groups) + 1)

In [34]:
proc_neighborhoods['id'] = range(1, len(neighbourhoods) + 1)
mapping = dict(proc_neighborhood_groups[['neighbourhood_group_cleansed', 'id']].values)
proc_neighborhoods['neighborhood_group'] = proc_neighborhoods['neighborhood_group'].map(mapping)

In [35]:
boolean_map = {'t': True, 'f': False}
proc_calendar['available'] = proc_calendar['available'].map(boolean_map)
proc_calendar['price'] = proc_calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['adjusted_price'] = proc_calendar['adjusted_price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['minimum_nights'] = proc_calendar['minimum_nights'].fillna(0).astype(int)
proc_calendar['maximum_nights'] = proc_calendar['maximum_nights'].fillna(0).astype(int)